In [18]:
%load_ext autoreload
%autoreload 2
import numpy as np
import xarray as xr
from matplotlib import pyplot as plt
import pandas as pd
import sys
sys.path.append('/home/onno/Thesis/Scripts')
import my_tools
from my_tools import plot_dic, file_dic
from mpl_toolkits.basemap import Basemap, addcyclic
from cmap import ncl_colormap

%matplotlib qt

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
#set path for general ERA5 reanalysis and GFS reforecast
path_gen = '/media/onno/Algemeen/Thesis/'
lat_0 = 52
lat_1 = 46
lon_0 = 6
lon_1 = 12
latlon = '_lon_{}_{}_lat_{}_{}'.format(lon_0,lon_1,lat_0,lat_1)

In [74]:
#Load ERA5 files 
era_env = xr.open_dataset(path_gen+'era51_mars_env_wledit2000-10000_latavg_v300_79-19_6hourly_anom_from_smoothed04_clim_smoothed.nc').squeeze()
era_t850 = xr.open_dataset(path_gen+'era51_mars_t850_79-19_6hourly_anom_from_smoothed04_clim.nc').squeeze()
era_cp = xr.open_dataset(path_gen+'era51_mars_phasevel_wledit2000-10000_latavg_v300_envgt15_79-19_6hourly_setvrange_-100to100.nc').squeeze()
era_v300 = xr.open_dataset(path_gen+'era51_mars_v300_wledit2000-10000_latavg_79-19_6hourly.nc').squeeze()

Make ERA5 composite Analysis

In [5]:
#set path for temp extremes files
path = '/media/onno/Algemeen/Thesis/GFS_T850/'

datez_pers_hw = np.load(path+'persistent_heatwaves'+latlon+'.npy')
datez_pers_cw = np.load(path+'persistent_coldwaves'+latlon+'.npy')
datez_short_hw = np.load(path+'short_heatwaves'+latlon+'.npy')
datez_short_cw = np.load(path+'short_coldwaves'+latlon+'.npy')
datez_total = [datez_pers_hw,datez_pers_cw,datez_short_hw,datez_short_cw]

ERA5 Composite analysis for RWP Envelope and 850 hPa Temperature

In [66]:
my_cmap = ncl_colormap()
titlez = ['Persistent Heatwaves','Persistent Coldwaves','Short-lived Heatwaves','Short-lived Coldwaves']
file_namez = ['persistent_heatwaves','persistent_coldwaves','short-heatwaves','short_coldwaves']
for i,datez in enumerate(datez_total):
    fig,ax = plt.subplots(figsize=(16,9))
    m = Basemap(projection='ortho', 
              lat_0=90, lon_0=0,
#               llcrnrlon=-60, 
#               llcrnrlat=25, 
#               urcrnrlon=60, 
#               urcrnrlat=75,
                ax=ax)
    era_env_sub = era_env.sel(time=datez[:,0]).mean(dim='time')
    era_env_sub_cyclic, lon_cyclic = addcyclic(era_env_sub.v.values, era_env_sub.longitude)
    era_t850_sub = era_t850.sel(time=datez[:,0]).mean(dim='time')
    era_t850_sub_cyclic, lon_cyclic = addcyclic(era_t850_sub.t.values, era_t850_sub.longitude)
    lon,lat = np.meshgrid(lon_cyclic,era_env_sub.latitude)
    im = m.contourf(lon,lat,era_env_sub_cyclic,latlon='true',cmap=my_cmap,
                   vmin=8,vmax=40,extend='both',levels=np.arange(8,41))
    cs = m.contour(lon,lat,era_t850_sub_cyclic,latlon='true',levels=[-10,-8,-6,-4,-2,2,4,6,8,10],
             colors='k')
    ax.clabel(cs,fmt='%d',colors='r',inline_spacing=.5)
#     m.drawcountries()
    m.drawcoastlines()
    m.drawparallels((-80,-60,-40,-20,0,20,40,60,80))
    m.drawmeridians([0,60,120,180,240,300],labels=[0,0,1,1])
    cbax = fig.add_axes([0.85, 0.2, 0.025, 0.6])
    fig.colorbar(im, cax=cbax)
    cbax.set_ylabel(plot_dic['envelope']['label'], rotation=90)
    fig.suptitle('GEFS RWP Envelope and 850 hPa Temp Composite Analysis for {}'.format(titlez[i]))
    fig.subplots_adjust(right=0.84)
    fig.savefig(path+'/composite/enn_t850{}_composite_{}'.format(latlon,file_namez[i]),dpi=200)

    
    


/home/onno/miniconda3/envs/thesis/lib/python3.8/site-packages/mpl_toolkits/basemap/__init__.py:5098: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return npsel.concatenate((a,a[slicer]),axis=axis)
/home/onno/miniconda3/envs/thesis/lib/python3.8/site-packages/mpl_toolkits/basemap/__init__.py:5098: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return npsel.concatenate((a,a[slicer]),axis=axis)
/home/onno/miniconda3/envs/thesis/lib/python3.8/site-packages/mpl_toolkits/basemap/__init__.py:5098: FutureWarning: Using a non-tuple sequence for mul

ERA5 composite analysis for RWP phase speed and meridional wind at 300 hPa

In [90]:
my_cmap = ncl_colormap()
titlez = ['Persistent Heatwaves','Persistent Coldwaves','Short-lived Heatwaves','Short-lived Coldwaves']
file_namez = ['persistent_heatwaves','persistent_coldwaves','short-heatwaves','short_coldwaves']
for i,datez in enumerate(datez_total):
    fig,ax = plt.subplots(figsize=(16,9))
    m = Basemap(projection='ortho', 
              lat_0=90, lon_0=0,
#               llcrnrlon=-60, 
#               llcrnrlat=25, 
#               urcrnrlon=60, 
#               urcrnrlat=75,
                ax=ax)
    era_cp_sub = era_cp.sel(time=datez[:,0]).mean(dim='time')
    era_cp_sub_cyclic, lon_cyclic = addcyclic(era_cp_sub.v.values, era_cp_sub.longitude)
    era_v300_sub = era_v300.sel(time=datez[:,0]).mean(dim='time')
    era_v300_sub_cyclic, lon_cyclic = addcyclic(era_v300_sub.v.values, era_v300_sub.longitude)
    lon,lat = np.meshgrid(lon_cyclic,era_cp_sub.latitude)
    im = m.contourf(lon,lat,era_cp_sub_cyclic,latlon='true',cmap=my_cmap,
                   vmin=0,vmax=10,extend='both',levels=np.arange(0,10.5,.5))
    cs = m.contour(lon,lat,era_v300_sub_cyclic,latlon='true',levels=[-20,-16,-12,-8,-4,-2,2,4,8,12,16,20],
             colors='k')
    ax.clabel(cs,fmt='%d',colors='r',inline_spacing=.5)
#     m.drawcountries()
    m.drawcoastlines()
    m.drawparallels((-80,-60,-40,-20,0,20,40,60,80))
    m.drawmeridians([0,60,120,180,240,300],labels=[0,0,1,1])
    cbax = fig.add_axes([0.85, 0.2, 0.025, 0.6])
    fig.colorbar(im, cax=cbax)
    cbax.set_ylabel(plot_dic['phasespeed']['label'], rotation=90)
    fig.suptitle('GEFS RWP Phase Speed and 300 hPa meridional wind Composite Analysis for {}'.format(titlez[i]))
    fig.subplots_adjust(right=0.84)
    fig.savefig(path+'/composite/phasespeed_v300{}_composite_{}'.format(latlon,file_namez[i]),dpi=200)
    plt.close(fig)

/home/onno/miniconda3/envs/thesis/lib/python3.8/site-packages/mpl_toolkits/basemap/__init__.py:5098: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return npsel.concatenate((a,a[slicer]),axis=axis)
/home/onno/miniconda3/envs/thesis/lib/python3.8/site-packages/mpl_toolkits/basemap/__init__.py:5098: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return npsel.concatenate((a,a[slicer]),axis=axis)
/home/onno/miniconda3/envs/thesis/lib/python3.8/site-packages/mpl_toolkits/basemap/__init__.py:5098: FutureWarning: Using a non-tuple sequence for mul